In [1]:
import pandas as pd
import numpy as np
from scipy.stats import skew

In [ ]:
def handle_outliers_revised(
    df: pd.DataFrame,
    feature_method_map: dict[str, str] = None,
    default_method: str = None, # 'auto', 'trim', 'winsorize', 'iqr', atau 'none'
    iqr_multiplier: float = 1.5,
    quantile_range: tuple[float, float] = (0.05, 0.95),
    min_data_threshold: int = 100,
    skew_threshold: float = 1.0
) -> pd.DataFrame:
    """
    Menangani outlier pada kolom numerik DataFrame dengan cerdas dan aman.

    Metode 'trim' dan 'iqr' mengidentifikasi semua baris outlier dari semua
    kolom terlebih dahulu, lalu menghapusnya dalam satu operasi untuk
    menjamin konsistensi hasil.

    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame input yang akan diproses.
    feature_method_map : dict[str, str], optional
        Peta untuk menerapkan metode spesifik per kolom, misal: {'Gaji': 'winsorize'}.
        Metode yang tersedia: 'trim', 'winsorize', 'iqr', 'auto', 'none'.
    default_method : str, optional
        Metode fallback jika tidak ada di peta. Jika None, akan disetel ke 'auto'.
    iqr_multiplier : float, default 1.5
        Pengali untuk metode 'iqr' dalam menentukan batas outlier.
    quantile_range : tuple[float, float], default (0.05, 0.95)
        Batas kuantil bawah dan atas untuk metode 'trim' dan 'winsorize'.
    min_data_threshold : int, default 100
        Batas jumlah data untuk penentuan metode otomatis. Di bawah ini, 'iqr' digunakan.
    skew_threshold : float, default 1.0
        Batas absolut skewness untuk penentuan metode otomatis.

    Returns:
    --------
    pd.DataFrame
        DataFrame baru yang telah ditangani outlier-nya.
    """
    df_out = df.copy()
    feature_method_map = feature_method_map or {}
    default_method = default_method or 'auto'
    
    indices_to_drop = set()

    # Fungsi helper untuk memilih metode otomatis
    def _choose_auto_method(col_data: pd.Series) -> str:
        if len(col_data.dropna()) < min_data_threshold:
            return 'iqr'
        elif abs(skew(col_data.dropna())) > skew_threshold:
            return 'winsorize'
        else:
            return 'trim'

    # Loop untuk menerapkan Winsorize dan mengumpulkan indeks untuk dihapus
    for col in df_out.select_dtypes(include=np.number).columns:
        method = feature_method_map.get(col, default_method)
        if method == 'auto':
            method = _choose_auto_method(df_out[col])
        
        if method == 'none':
            continue

        # Hitung batas berdasarkan metode
        lower_bound, upper_bound = None, None
        if method in ['trim', 'winsorize']:
            q_low, q_high = df_out[col].quantile(quantile_range)
            lower_bound, upper_bound = q_low, q_high
        elif method == 'iqr':
            Q1 = df_out[col].quantile(0.25)
            Q3 = df_out[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - iqr_multiplier * IQR
            upper_bound = Q3 + iqr_multiplier * IQR

        # Lakukan aksi
        if method == 'winsorize':
            df_out[col] = np.clip(df_out[col], lower_bound, upper_bound)
        elif method in ['trim', 'iqr']:
            # Kumpulkan indeks baris yang merupakan outlier
            outlier_indices = df_out.index[(df_out[col] < lower_bound) | (df_out[col] > upper_bound)]
            indices_to_drop.update(outlier_indices)
        elif method != 'none':
            raise ValueError(f"Metode '{method}' untuk kolom '{col}' tidak dikenali.")

    # Hapus semua baris outlier yang terkumpul dalam satu langkah
    if indices_to_drop:
        df_out.drop(index=list(indices_to_drop), inplace=True)
        
    return df_out

# Contoh Implementasi

In [3]:
# Membuat DataFrame contoh
data = {
    'Gaji': [5000, 6000, 5500, 7000, 6500, 5800, 7200, 6300, 5900, 500000], # Sangat skewed
    'Skor_Tes': [88, 92, 85, 95, 90, 89, 93, 87, 5, 105], # Simetris dengan outlier
    'Jumlah_Anak': [1, 2, 2, 1, 3, 2, 8, 1, 2, 1], # Data kecil, metode IQR cocok
    'Departemen': ['A', 'B', 'A', 'C', 'B', 'A', 'C', 'A', 'B', 'A'] # Non-numerik
}
df_contoh = pd.DataFrame(data)

print("Data Asli:")
print(df_contoh)

Data Asli:
     Gaji  Skor_Tes  Jumlah_Anak Departemen
0    5000        88            1          A
1    6000        92            2          B
2    5500        85            2          A
3    7000        95            1          C
4    6500        90            3          B
5    5800        89            2          A
6    7200        93            8          C
7    6300        87            1          A
8    5900         5            2          B
9  500000       105            1          A


<h2> Mode Otomatis (Penggunaan Default)

In [4]:
df_bersih_auto = handle_outliers_revised(df_contoh)
print("\nContoh 1: Mode Otomatis")
print(df_bersih_auto)


Contoh 1: Mode Otomatis
   Gaji  Skor_Tes  Jumlah_Anak Departemen
0  5000        88            1          A
1  6000        92            2          B
2  5500        85            2          A
3  7000        95            1          C
4  6500        90            3          B
5  5800        89            2          A
7  6300        87            1          A


<h2> Metode Spesifik per Kolom (feature_method_map)

In [5]:
peta_kustom = {
    'Gaji': 'trim',          # Paksa hapus outlier Gaji, jangan di-winsorize
    'Skor_Tes': 'none'       # Jangan lakukan apa-apa pada Skor_Tes
}
df_bersih_map = handle_outliers_revised(df_contoh, feature_method_map=peta_kustom)
print("\nContoh 2: Metode Spesifik per Kolom")
print(df_bersih_map)


Contoh 2: Metode Spesifik per Kolom
   Gaji  Skor_Tes  Jumlah_Anak Departemen
1  6000        92            2          B
2  5500        85            2          A
3  7000        95            1          C
4  6500        90            3          B
5  5800        89            2          A
7  6300        87            1          A
8  5900         5            2          B


<h2> Mengubah Metode Default (default_method)

In [6]:
df_bersih_default = handle_outliers_revised(df_contoh, default_method='winsorize')
print("\nContoh 3: Mengubah Metode Default menjadi 'winsorize'")
print(df_bersih_default)


Contoh 3: Mengubah Metode Default menjadi 'winsorize'
     Gaji  Skor_Tes  Jumlah_Anak Departemen
0    5225      88.0         1.00          A
1    6000      92.0         2.00          B
2    5500      85.0         2.00          A
3    7000      95.0         1.00          C
4    6500      90.0         3.00          B
5    5800      89.0         2.00          A
6    7200      93.0         5.75          C
7    6300      87.0         1.00          A
8    5900      41.0         2.00          B
9  278240     100.5         1.00          A


d:\ANACONDA\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all-round floats to integers. Instead call result.infer_objects(copy=False) for object inference, or cast round floats explicitly. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return bound(*args, **kwds)


<h2> Mengatur Parameter Kuantil & IQR

In [7]:
# Membuat deteksi lebih ketat (hanya 1% data teratas/terbawah dianggap outlier)
# dan rentang IQR lebih longgar (lebih toleran terhadap outlier)
df_bersih_param = handle_outliers_revised(
    df_contoh,
    quantile_range=(0.01, 0.99),
    iqr_multiplier=2.5
)
print("\nContoh 4: Mengatur Parameter Kuantil & IQR")
print(df_bersih_param)


Contoh 4: Mengatur Parameter Kuantil & IQR
   Gaji  Skor_Tes  Jumlah_Anak Departemen
0  5000        88            1          A
1  6000        92            2          B
2  5500        85            2          A
3  7000        95            1          C
4  6500        90            3          B
5  5800        89            2          A
7  6300        87            1          A


<h2> Mengatur Parameter Mode Otomatis

In [8]:
# Anggap kolom 'skewed' jika skew > 0.8 (lebih sensitif)
# Anggap data 'kecil' jika < 15 baris
df_bersih_auto_param = handle_outliers_revised(
    df_contoh,
    skew_threshold=0.8,
    min_data_threshold=15
)
print("\nContoh 5: Mengatur Parameter Mode Otomatis")
print(df_bersih_auto_param)


Contoh 5: Mengatur Parameter Mode Otomatis
   Gaji  Skor_Tes  Jumlah_Anak Departemen
0  5000        88            1          A
1  6000        92            2          B
2  5500        85            2          A
3  7000        95            1          C
4  6500        90            3          B
5  5800        89            2          A
7  6300        87            1          A


<h1> Contoh Implementasi dengan Dataset > 100 baris

In [9]:
# Membuat dataset yang lebih besar
np.random.seed(42) # Agar hasilnya selalu sama
data_normal = np.random.randn(200) * 10 + 75 # Rata-rata 75
data_normal[198] = 200 # Outlier atas
data_normal[199] = -50 # Outlier bawah

data_miring = np.random.gamma(2, size=200) * 1000
data_miring[199] = 90000 # Outlier ekstrem

df_besar = pd.DataFrame({
    'Distribusi_Normal': data_normal,
    'Distribusi_Miring': data_miring
})

print("Karakteristik Data Besar (Sebelum):")
print(df_besar.describe())
print("\nSkewness (Sebelum):")
print(df_besar.skew())

Karakteristik Data Besar (Sebelum):
       Distribusi_Normal  Distribusi_Miring
count         200.000000         200.000000
mean           74.646528        2452.868088
std            15.591514        6380.026947
min           -50.000000         153.513495
25%            67.948723         958.992279
50%            74.958081        1688.453064
75%            80.154360        2649.037721
max           200.000000       90000.000000

Skewness (Sebelum):
Distribusi_Normal     0.070857
Distribusi_Miring    13.117854
dtype: float64


In [10]:
# Menjalankan fungsi pada data besar
df_besar_bersih = handle_outliers_revised(df_besar)

print("\nKarakteristik Data Besar (Sesudah):")
print(df_besar_bersih.describe())
print("\nSkewness (Sesudah):")
print(df_besar_bersih.skew())
print(f"\nJumlah baris sebelum: {len(df_besar)}")
print(f"Jumlah baris sesudah: {len(df_besar_bersih)}")


Karakteristik Data Besar (Sesudah):
       Distribusi_Normal  Distribusi_Miring
count         180.000000         180.000000
mean           74.553164        1979.524840
std             7.504460        1309.030889
min            59.851528         407.413961
25%            68.990943         959.686678
50%            74.958081        1662.175758
75%            79.733263        2532.984402
max            90.499344        5125.559058

Skewness (Sesudah):
Distribusi_Normal    0.017568
Distribusi_Miring    1.005164
dtype: float64

Jumlah baris sebelum: 200
Jumlah baris sesudah: 180
